In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dat
import matplotlib as mplt
import datetime as dt
import xgboost as xgb
%matplotlib inline
sns.set()

In [2]:
auctions = pd.read_csv("data/auctions.csv", dtype={'date': 'str','device_id': 'category', 'ref_type_id': 'category','source_id':'category'}, parse_dates = ['date'])
auctions.head(5)

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [3]:
auctions=auctions.sort_values(by='date')

In [4]:
serie=auctions.groupby('device_id').cumcount()
serie=serie.to_list()
auctions['entradas_previas']=serie

In [5]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')
auctions_ventana1=auctions.loc[auctions['date']<fecha_tope].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana1['tiempo_hasta_aparicion']=(auctions_ventana1['date']-fecha_minima)/ np.timedelta64(1, 's')

In [6]:
auctions_ventana1_reducido=auctions_ventana1.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana1_reducido=auctions_ventana1_reducido.dropna()

In [7]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')

auctions_ventana2=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana2['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(1, 's')

In [8]:
auctions_ventana2_reducido=auctions_ventana2.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana2_reducido=auctions_ventana2_reducido.dropna()

In [9]:
fecha_minima=pd.to_datetime('2019-04-22 00:00:00')
fecha_tope=pd.to_datetime('2019-04-25 00:00:00')

auctions_ventana3=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana3['tiempo_hasta_aparicion']=(auctions_ventana3['date']-fecha_minima)/ np.timedelta64(1, 's')
auctions_ventana3['dia_mean']=auctions_ventana3['date'].dt.day
auctions_ventana3['dia_desvio_std']=auctions_ventana3['date'].dt.day
auctions_ventana3.head(25)
auctions_ventana3_reducido=auctions_ventana3.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana3_reducido=auctions_ventana3_reducido.dropna()

In [10]:
fecha_minima=pd.to_datetime('2019-04-23 00:00:00')
fecha_tope=pd.to_datetime('2019-04-26 00:00:00')

auctions_ventana4=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana4['tiempo_hasta_aparicion']=(auctions_ventana4['date']-fecha_minima)/ np.timedelta64(1, 's')
auctions_ventana4['dia_mean']=auctions_ventana4['date'].dt.day
auctions_ventana4['dia_desvio_std']=auctions_ventana4['date'].dt.day
auctions_ventana4.head(25)
auctions_ventana4_reducido=auctions_ventana4.groupby(['device_id']).agg({"date" : min,\
                                                                        # "std_hour" : "first",\
                                                                        # "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana4_reducido=auctions_ventana4_reducido.dropna()

In [19]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')

auctions_ventana5=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana5['tiempo_hasta_aparicion']=(auctions_ventana5['date']-fecha_minima)/ np.timedelta64(1, 's')
auctions_ventana5['dia_mean']=auctions_ventana5['date'].dt.day
auctions_ventana5['dia_desvio_std']=auctions_ventana5['date'].dt.day
auctions_ventana5.head(25)
auctions_ventana5_reducido=auctions_ventana5.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana5_reducido=auctions_ventana5_reducido.dropna()

In [12]:
auctions_ventana2_reducido.head()

,device_id,date,tiempo_hasta_aparicion,entradas_previas
0,100033926124811452,2019-04-23 03:10:50.587009,184250.587009,0.0
1,1000400432115255220,2019-04-21 02:41:03.200096,9663.200096,1.0
2,1001123163431776865,2019-04-21 01:41:47.707241,6107.707241,68.0
3,1001144380199556647,2019-04-21 15:12:55.733409,54775.733409,78.0
4,1001358436431521709,2019-04-21 03:39:07.387349,13147.387349,0.0


In [13]:
competencia = pd.read_csv("data/target_competencia_ids.csv", dtype={'ref_hash': 'category'}) 

In [14]:
auctions_ventana5_reducido.head()

,device_id,date,tiempo_hasta_aparicion,entradas_previas
1,1000400432115255220,2019-04-24 22:18:41.231385,80321.231385,40.0
2,1001123163431776865,2019-04-24 01:34:20.785346,5660.785346,166.0
3,1001144380199556647,2019-04-24 02:52:27.414060,10347.414060,257.0
4,1001358436431521709,2019-04-24 01:08:48.217577,4128.217577,62.0
5,1001650136929210538,2019-04-24 00:22:27.574516,1347.574516,2138.0


In [20]:
competencia[['ref_hash','prediccion']] = competencia['ref_hash'].str.split('_',expand=True)
competencia.head()

In [21]:
competencia=competencia.loc[competencia['prediccion']=='sc']

In [22]:
competencia.head()

,ref_hash,obj,prediccion
0,1000169251625791246,0,sc
2,1000395625957344683,0,sc
4,1003027494996471685,0,sc
6,1006670001679961544,0,sc
8,1007573308966476713,0,sc


In [23]:
auctions_ventana5_reducido=auctions_ventana5_reducido.merge(competencia,how='right',left_on='device_id',right_on='ref_hash')

In [24]:
auctions_ventana5_reducido.head()

,device_id,date,tiempo_hasta_aparicion,entradas_previas,ref_hash,obj,prediccion
0,1026825193119998092,2019-04-24 06:20:42.709101,22842.709101,4.0,1026825193119998092,0,sc
1,1032938399279723623,2019-04-24 02:58:32.169083,10712.169083,113.0,1032938399279723623,0,sc
2,1071701264087131067,2019-04-24 00:00:23.704470,23.704470,107.0,1071701264087131067,0,sc
3,1083324672128707147,2019-04-24 04:52:51.179495,17571.179495,50.0,1083324672128707147,0,sc
4,109389620877102250,2019-04-24 04:28:55.803587,16135.803587,373.0,109389620877102250,0,sc


In [25]:
auctions_ventana5_reducido['tiempo_hasta_aparicion']=auctions_ventana5_reducido['tiempo_hasta_aparicion'].fillna(259200)#3 dias en segundos

In [26]:
auctions_ventana5_reducido.head()

,device_id,date,tiempo_hasta_aparicion,entradas_previas,ref_hash,obj,prediccion
0,1026825193119998092,2019-04-24 06:20:42.709101,22842.709101,4.0,1026825193119998092,0,sc
1,1032938399279723623,2019-04-24 02:58:32.169083,10712.169083,113.0,1032938399279723623,0,sc
2,1071701264087131067,2019-04-24 00:00:23.704470,23.704470,107.0,1071701264087131067,0,sc
3,1083324672128707147,2019-04-24 04:52:51.179495,17571.179495,50.0,1083324672128707147,0,sc
4,109389620877102250,2019-04-24 04:28:55.803587,16135.803587,373.0,109389620877102250,0,sc


In [27]:
auctions_ventana5_reducido=auctions_ventana5_reducido.drop(['date','obj','prediccion'],axis=1)

In [28]:
auctions_ventana5_reducido['device_id']=auctions_ventana5_reducido['device_id'].fillna(auctions_ventana5_reducido['ref_hash'])

In [29]:
auctions_ventana5_reducido=auctions_ventana5_reducido.set_index('device_id')

In [33]:
#target

auctions_ventana5_reducido=auctions_ventana5_reducido.drop(['ref_hash'],axis=1)
auctions_ventana5_reducido.head()

KeyError: "['ref_hash'] not found in axis"

In [31]:

auctions_ventana2_reducido.head()

,device_id,date,tiempo_hasta_aparicion,entradas_previas
0,100033926124811452,2019-04-23 03:10:50.587009,184250.587009,0.0
1,1000400432115255220,2019-04-21 02:41:03.200096,9663.200096,1.0
2,1001123163431776865,2019-04-21 01:41:47.707241,6107.707241,68.0
3,1001144380199556647,2019-04-21 15:12:55.733409,54775.733409,78.0
4,1001358436431521709,2019-04-21 03:39:07.387349,13147.387349,0.0


In [32]:
auctions_ventana2_reducido=auctions_ventana2_reducido.drop(['date'],axis=1)
auctions_ventana2_reducido.head()

,device_id,tiempo_hasta_aparicion,entradas_previas
0,100033926124811452,184250.587009,0.0
1,1000400432115255220,9663.200096,1.0
2,1001123163431776865,6107.707241,68.0
3,1001144380199556647,54775.733409,78.0
4,1001358436431521709,13147.387349,0.0
5,1001650136929210538,48.667344,1479.0
6,1001893648440705480,146533.939844,73.0
7,1002004782572506858,12671.761233,150.0
8,1002085736203824817,4118.989811,8.0
9,1002447145227437936,3465.745162,550.0


In [36]:
#trainear con esto ESTOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

columnsTitles=['device_id','entradas_previas','tiempo_hasta_aparicion']
auctions_ventana2_reducido=auctions_ventana2_reducido.reindex(columns=columnsTitles)
auctions_ventana2_reducido.head()

,device_id,entradas_previas,tiempo_hasta_aparicion
0,100033926124811452,0.0,184250.587009
1,1000400432115255220,1.0,9663.200096
2,1001123163431776865,68.0,6107.707241
3,1001144380199556647,78.0,54775.733409
4,1001358436431521709,0.0,13147.387349


In [37]:
#TESTEAR CON ESTO

columnsTitles=['device_id','entradas_previas','tiempo_hasta_aparicion']
auctions_ventana5_reducido=auctions_ventana5_reducido.reindex(columns=columnsTitles)
auctions_ventana5_reducido.head()

,device_id,entradas_previas,tiempo_hasta_aparicion
device_id,,,
1026825193119998092,NaN,4.0,22842.709101
1032938399279723623,NaN,113.0,10712.169083
1071701264087131067,NaN,107.0,23.704470
1083324672128707147,NaN,50.0,17571.179495
109389620877102250,NaN,373.0,16135.803587
